In [1]:
import numpy as np
from sys import maxsize
import random

*R* : Set of *m* resources represented as (id, cost?) tuples

In [2]:
m = 10

R = []
R = [(i, np.random.randint(0, 100)) for i in range(m)]
print(R)

[(0, 68), (1, 55), (2, 61), (3, 5), (4, 53), (5, 49), (6, 35), (7, 53), (8, 34), (9, 21)]


*N* : Set of *n* agents as (id, balance?, priority?)

In [3]:
n = 2

N = []
N = [(i, np.random.randint(0, 100), np.random.randint(0, 100)) for i in range(n)]
N

[(0, 76, 80), (1, 80, 52)]

Zero Allocations

In [4]:
def init_allocs(resources, agents):
    allocs = []
    for index, i in enumerate(agents):
        r_i = [0 for x in resources]
        allocs.append(r_i)
        
    return allocs

A = init_allocs(R, N)
print(A)

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


Random allocations

In [5]:
def rand_allocs(resources, agents):
    allocs = init_allocs(resources, agents)
    for index, i in enumerate(resources):
        allocs[np.random.randint(0, 100)//(100//len(agents))][index] = 1
    return allocs

In [6]:
A = rand_allocs(R, N)
print(A)

[[1, 1, 0, 0, 1, 1, 0, 0, 1, 0], [0, 0, 1, 1, 0, 0, 1, 1, 0, 1]]


Random utility values of resources for both users : U

In [7]:
U = []

for index, i in enumerate(N):
    u_i = random.sample(range(0, 100), len(R))
    U.append(u_i)
    
    
print(U)

[[48, 74, 64, 20, 79, 2, 28, 54, 4, 18], [37, 14, 75, 90, 16, 26, 98, 88, 50, 49]]


Random pair-wise complementing utility values : V

In [8]:
V = []

for index, i in enumerate(N):
    v_ijjp1 = random.sample(range(0, 100), len(R)//2)
    V.append(v_ijjp1)
    
    
print(V)

[[27, 69, 48, 76, 59], [98, 26, 3, 31, 43]]


In [9]:
additive_utililty = lambda alloc, utility : sum(np.array(alloc) * np.array(utility))

def complementing_utility(alloc, utility, p_utility) :
    pairwise_alloc = []
    for i in range(0, len(alloc), 2):
        pairwise_alloc.append(alloc[i] & alloc[i+1])
        
#     print(alloc)
#     print(pairwise_alloc)
#     print(utility)
#     print(p_utility)
    return additive_utililty(alloc, utility) + additive_utililty(pairwise_alloc, p_utility)

In [36]:
print(additive_utililty(A[0], U[0]))
print(complementing_utility(A[0], U[0], V[0]))

207
282


In [34]:
V[0]

[27, 69, 48, 76, 59]

In [11]:
def optimal_nash_prod_wcomp(utility, p_utility) :
    n = len(utility)
    if n :
        m = len(utility[0])
        
    optimal_alloc = np.zeros((n, m))
    opt_payoff = 0
    for i in range(2**(m*n)) :
        curr_alloc = np.array([int(x) for x in list(np.binary_repr(i, width=(m*n)))]).reshape(n, m).tolist()
        req_resources = np.zeros(m)
        payoff = 1
        # print(curr_alloc)
        for j in range(len(curr_alloc)) :
            req_resources += curr_alloc[j]
        if not np.array_equal(req_resources, np.ones(m)) :
            continue
        for j in range(len(curr_alloc)) :
            payoff *= complementing_utility(curr_alloc[j], utility[j], p_utility[j])
            
        if opt_payoff < payoff :
            opt_payoff = payoff
            optimal_alloc = curr_alloc
            
    return(optimal_alloc)

In [12]:
onpa = optimal_nash_prod_wcomp(U, V)

<hr>
#### Envy and envy up to one good calculations
<hr>

In [52]:
def envyMap(allocs, utilities, p_utilities):
    size_agents = np.array(allocs).shape[0]
    envy_map = np.zeros((size_agents, size_agents))
    for i in range(len(envy_map)):
        for j in range(len(envy_map[i])):
            envy_map[i][j] = complementing_utility(allocs[i], utilities[i], p_utilities[i]) \
            - complementing_utility(allocs[j], utilities[j], p_utilities[j])
    return envy_map

In [53]:
def envyMap_upto1(allocs, utilities, p_utilities):
    size_agents = np.array(allocs).shape[0]
    size_res = np.array(allocs).shape[1]
    envy_map = np.zeros((size_agents, size_agents))
    for i in range(len(envy_map)) :
        for j in range(len(envy_map[i])):
            envy_map[i][j] = complementing_utility(allocs[i], utilities[i], p_utilities[i]) \
            - complementing_utility(allocs[j], utilities[i], p_utilities[i])
            if envy_map[i][j] < 0 :
                envy_res_drop1_j = np.zeros(size_res)
                max_pos_envy = - maxsize
                for x in range(len(envy_res_drop1_j)) :
                    new_alloc_j = list(allocs[j])
                    new_alloc_j[x] = 0
                    envy_res_drop1_j[x] = complementing_utility(allocs[i], utilities[i], p_utilities[i]) \
                    - complementing_utility(new_alloc_j, utilities[i], p_utilities[i])
                    # print(envy_res_drop1_j)
                    if envy_res_drop1_j[x] >= max_pos_envy:
                        envy_map[i][j] = envy_res_drop1_j[x]
                        max_pos_envy = envy_res_drop1_j[x]
                
    return envy_map

In [51]:
print(U)
print(V)

[[48, 74, 64, 20, 79, 2, 28, 54, 4, 18], [37, 14, 75, 90, 16, 26, 98, 88, 50, 49]]
[[27, 69, 48, 76, 59], [98, 26, 3, 31, 43]]


Envy map represents the envy between two agents. $envymap_{ij}$ is the amount with which i envies j

In [62]:
rand_a = rand_allocs(R, N)
print("Random allocations : ", rand_a)
print(envyMap(rand_a, U, V))
print(envyMap_upto1(rand_a, U, V))

Random allocations :  [[0, 0, 1, 1, 1, 1, 0, 1, 0, 0], [1, 1, 0, 0, 0, 0, 1, 0, 1, 1]]
[[  0. -53.]
 [ 53.   0.]]
[[ 0. 78.]
 [65.  0.]]


In [63]:
print("Nash product allocations : ", onpa)
print(envyMap(onpa, U, V))
print(envyMap_upto1(onpa, U, V))

Nash product allocations :  [[1, 1, 1, 1, 1, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 1, 1, 1]]
[[  0.  72.]
 [-72.   0.]]
[[  0. 192.]
 [109.   0.]]


<hr>
#### Envy free experiments on large sample
<hr>

m => Number of resources <br>
R => Resources <id, value> <br>
n => Number of agents <br>
N => Agents <id, value, priority> <br>
U => Additive utilities
V => Complementing Utilities

Additive utilities

In [108]:
np.random.seed(10)

ef1_tracker = []

for x in range(1000):
    m = 10
    R = []
    R = [(i, np.random.randint(0, 100)) for i in range(m)]

    n = 2
    N = []
    N = [(i, np.random.randint(0, 100), np.random.randint(0, 100)) for i in range(n)]

    U = []
    for index, i in enumerate(N):
        u_i = np.random.random_sample(len(R)) * 100
        U.append(u_i)

    V = []
    for index, i in enumerate(N):
        v_ijjp1 = np.random.random_sample(len(R)//2) * 0
        V.append(v_ijjp1)

    random_alloc = rand_allocs(R, N)
    envymap1 = envyMap_upto1(random_alloc, U, V)
    ef1_tracker.append(not bool(np.sum(envymap1<0)))
    
    print("Allocation : ", random_alloc)
    print(envymap1)
    print("Is EF1 allocation? ", ef1_tracker[x])
    
print("EF1 percent : ", np.sum(ef1_tracker)/len(ef1_tracker) * 100)

Allocation :  [[1, 1, 0, 0, 0, 0, 0, 1, 1, 0], [0, 0, 1, 1, 1, 1, 1, 0, 0, 1]]
[[  0.         -70.45463552]
 [128.21670918   0.        ]]
Is EF1 allocation?  False
Allocation :  [[0, 0, 1, 0, 0, 0, 1, 1, 1, 1], [1, 1, 0, 1, 1, 1, 0, 0, 0, 0]]
[[  0.          48.69098862]
 [-13.9941459    0.        ]]
Is EF1 allocation?  False
Allocation :  [[1, 1, 0, 1, 1, 0, 0, 1, 1, 0], [0, 0, 1, 0, 0, 1, 1, 0, 0, 1]]
[[  0.          38.13811332]
 [-97.98335624   0.        ]]
Is EF1 allocation?  False
Allocation :  [[1, 1, 0, 0, 1, 1, 0, 0, 1, 1], [0, 0, 1, 1, 0, 0, 1, 1, 0, 0]]
[[  0.         126.45461822]
 [-19.60666429   0.        ]]
Is EF1 allocation?  False
Allocation :  [[1, 1, 0, 1, 0, 1, 1, 0, 0, 0], [0, 0, 1, 0, 1, 0, 0, 1, 1, 1]]
[[  0.         -34.90561169]
 [  5.47709896   0.        ]]
Is EF1 allocation?  False
Allocation :  [[0, 1, 0, 0, 0, 0, 1, 1, 0, 1], [1, 0, 1, 1, 1, 1, 0, 0, 1, 0]]
[[  0.         -53.68527143]
 [-11.96388369   0.        ]]
Is EF1 allocation?  False
Allocation :  [[

 [-42.26123017   0.        ]]
Is EF1 allocation?  False
Allocation :  [[0, 1, 0, 1, 1, 1, 0, 1, 1, 1], [1, 0, 1, 0, 0, 0, 1, 0, 0, 0]]
[[   0.          316.24220381]
 [-222.340493      0.        ]]
Is EF1 allocation?  False
Allocation :  [[1, 1, 1, 1, 0, 1, 1, 1, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 1, 1]]
[[   0.          354.47786468]
 [-436.29641305    0.        ]]
Is EF1 allocation?  False
Allocation :  [[1, 1, 1, 0, 1, 0, 1, 0, 0, 1], [0, 0, 0, 1, 0, 1, 0, 1, 1, 0]]
[[  0.         112.67488487]
 [-61.3681435    0.        ]]
Is EF1 allocation?  False
Allocation :  [[1, 0, 0, 0, 0, 0, 0, 1, 1, 0], [0, 1, 1, 1, 1, 1, 1, 0, 0, 1]]
[[   0.         -119.72791419]
 [ 191.62341432    0.        ]]
Is EF1 allocation?  False
Allocation :  [[0, 0, 0, 0, 0, 1, 1, 1, 0, 1], [1, 1, 1, 1, 1, 0, 0, 0, 1, 0]]
[[  0.          94.26572629]
 [125.97217166   0.        ]]
Is EF1 allocation?  True
Allocation :  [[0, 1, 0, 0, 0, 1, 1, 0, 1, 0], [1, 0, 1, 1, 1, 0, 0, 1, 0, 1]]
[[ 0.         69.65881014]
 [ 2.09

Allocation :  [[1, 1, 1, 0, 0, 1, 1, 1, 1, 1], [0, 0, 0, 1, 1, 0, 0, 0, 0, 0]]
[[   0.          303.66635415]
 [-232.03355612    0.        ]]
Is EF1 allocation?  False
Allocation :  [[1, 1, 1, 1, 0, 1, 1, 1, 1, 1], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]]
[[   0.          509.32767508]
 [-304.55360685    0.        ]]
Is EF1 allocation?  False
Allocation :  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 0]]
[[   0.         -314.73758543]
 [ 494.61810339    0.        ]]
Is EF1 allocation?  False
Allocation :  [[0, 1, 1, 0, 0, 1, 0, 1, 0, 1], [1, 0, 0, 1, 1, 0, 1, 0, 1, 0]]
[[ 0.         36.56266725]
 [54.51505722  0.        ]]
Is EF1 allocation?  True
Allocation :  [[1, 1, 0, 0, 0, 0, 1, 1, 1, 1], [0, 0, 1, 1, 1, 1, 0, 0, 0, 0]]
[[  0.         149.79863808]
 [ 78.08852107   0.        ]]
Is EF1 allocation?  True
Allocation :  [[1, 0, 1, 1, 1, 1, 1, 0, 0, 1], [0, 1, 0, 0, 0, 0, 0, 1, 1, 0]]
[[  0.         146.35640474]
 [-52.03082126   0.        ]]
Is EF1 allocation?  False
Allocation

Is EF1 allocation?  False
Allocation :  [[0, 0, 1, 0, 0, 0, 0, 0, 1, 0], [1, 1, 0, 1, 1, 1, 1, 1, 0, 1]]
[[  0.         -47.60891665]
 [230.76029477   0.        ]]
Is EF1 allocation?  False
Allocation :  [[1, 1, 1, 0, 0, 1, 1, 0, 0, 1], [0, 0, 0, 1, 1, 0, 0, 1, 1, 0]]
[[ 0.         75.19468742]
 [33.61473357  0.        ]]
Is EF1 allocation?  True
Allocation :  [[1, 1, 1, 1, 1, 0, 1, 0, 1, 0], [0, 0, 0, 0, 0, 1, 0, 1, 0, 1]]
[[  0.         100.93917988]
 [-52.38834073   0.        ]]
Is EF1 allocation?  False
Allocation :  [[0, 1, 0, 1, 0, 0, 1, 1, 0, 1], [1, 0, 1, 0, 1, 1, 0, 0, 1, 0]]
[[   0.         -129.90557408]
 [  46.87431065    0.        ]]
Is EF1 allocation?  False
Allocation :  [[0, 1, 0, 1, 0, 1, 0, 0, 1, 0], [1, 0, 1, 0, 1, 0, 1, 1, 0, 1]]
[[  0.          28.47786873]
 [142.07965684   0.        ]]
Is EF1 allocation?  True
Allocation :  [[0, 1, 0, 1, 1, 0, 1, 1, 0, 0], [1, 0, 1, 0, 0, 1, 0, 0, 1, 1]]
[[  0.         111.40077514]
 [-31.5569128    0.        ]]
Is EF1 allocation?

Allocation :  [[0, 0, 0, 1, 0, 1, 1, 1, 0, 0], [1, 1, 1, 0, 1, 0, 0, 0, 1, 1]]
[[  0.          55.15631109]
 [125.25743623   0.        ]]
Is EF1 allocation?  True
Allocation :  [[1, 0, 1, 0, 1, 0, 1, 0, 1, 0], [0, 1, 0, 1, 0, 1, 0, 1, 0, 1]]
[[ 0.         -9.18929261]
 [-0.87019736  0.        ]]
Is EF1 allocation?  False
Allocation :  [[0, 0, 1, 0, 0, 1, 0, 0, 0, 0], [1, 1, 0, 1, 1, 0, 1, 1, 1, 1]]
[[   0.         -276.75227973]
 [ 207.64203648    0.        ]]
Is EF1 allocation?  False
Allocation :  [[0, 1, 0, 0, 1, 0, 0, 0, 0, 1], [1, 0, 1, 1, 0, 1, 1, 1, 1, 0]]
[[  0.         -17.82713543]
 [378.42732668   0.        ]]
Is EF1 allocation?  False
Allocation :  [[0, 1, 0, 1, 0, 1, 0, 1, 0, 0], [1, 0, 1, 0, 1, 0, 1, 0, 1, 1]]
[[  0.          14.63438156]
 [150.56657189   0.        ]]
Is EF1 allocation?  True
Allocation :  [[1, 0, 1, 0, 1, 1, 1, 0, 0, 0], [0, 1, 0, 1, 0, 0, 0, 1, 1, 1]]
[[  0.         -88.52791556]
 [ 24.49292426   0.        ]]
Is EF1 allocation?  False
Allocation :  [[0,

Allocation :  [[0, 1, 0, 0, 1, 1, 1, 1, 1, 1], [1, 0, 1, 1, 0, 0, 0, 0, 0, 0]]
[[   0.          313.22421745]
 [-304.31340682    0.        ]]
Is EF1 allocation?  False
Allocation :  [[0, 0, 0, 1, 1, 1, 1, 0, 0, 0], [1, 1, 1, 0, 0, 0, 0, 1, 1, 1]]
[[ 0.         21.86618539]
 [82.81717651  0.        ]]
Is EF1 allocation?  True
Allocation :  [[1, 0, 0, 0, 0, 0, 0, 1, 1, 0], [0, 1, 1, 1, 1, 1, 1, 0, 0, 1]]
[[  0.          45.56591102]
 [199.70717593   0.        ]]
Is EF1 allocation?  True
Allocation :  [[0, 1, 0, 0, 1, 0, 1, 0, 1, 0], [1, 0, 1, 1, 0, 1, 0, 1, 0, 1]]
[[ 0.         36.72911944]
 [98.61005102  0.        ]]
Is EF1 allocation?  True
Allocation :  [[1, 1, 0, 0, 1, 1, 0, 0, 1, 1], [0, 0, 1, 1, 0, 0, 1, 1, 0, 0]]
[[  0.         101.50784377]
 [ 47.35770791   0.        ]]
Is EF1 allocation?  True
Allocation :  [[0, 1, 1, 1, 1, 0, 0, 0, 0, 1], [1, 0, 0, 0, 0, 1, 1, 1, 1, 0]]
[[  0.          85.68445934]
 [198.0555394    0.        ]]
Is EF1 allocation?  True
Allocation :  [[1, 0, 1, 

Complementing utilities

In [109]:
np.random.seed(10)
for x in range(1000):
    m = 10
    R = []
    R = [(i, np.random.randint(0, 100)) for i in range(m)]

    n = 2
    N = []
    N = [(i, np.random.randint(0, 100), np.random.randint(0, 100)) for i in range(n)]

    U = []
    for index, i in enumerate(N):
        u_i = np.random.random_sample(len(R)) * 100
        U.append(u_i)

    V = []
    for index, i in enumerate(N):
        v_ijjp1 = np.random.random_sample(len(R)//2) * 100
        V.append(v_ijjp1)

    random_alloc = rand_allocs(R, N)
    envymap1 = envyMap_upto1(random_alloc, U, V)
    ef1_tracker.append(not bool(np.sum(envymap1<0)))
    
    print("Allocation : ", random_alloc)
    print(envymap1)
    print("Is EF1 allocation? ", ef1_tracker[x])
    
print("EF1 percent : ", np.sum(ef1_tracker)/len(ef1_tracker) * 100)

Allocation :  [[1, 1, 0, 0, 0, 0, 0, 1, 1, 0], [0, 0, 1, 1, 1, 1, 1, 0, 0, 1]]
[[  0.         -47.57696155]
 [240.91440766   0.        ]]
Is EF1 allocation?  False
Allocation :  [[0, 0, 1, 0, 0, 0, 1, 1, 1, 1], [1, 1, 0, 1, 1, 1, 0, 0, 0, 0]]
[[  0.          67.17398221]
 [-62.74625893   0.        ]]
Is EF1 allocation?  False
Allocation :  [[1, 1, 0, 1, 1, 0, 0, 1, 1, 0], [0, 0, 1, 0, 0, 1, 1, 0, 0, 1]]
[[   0.           59.20772273]
 [-162.77259922    0.        ]]
Is EF1 allocation?  False
Allocation :  [[1, 1, 0, 0, 1, 1, 0, 0, 1, 1], [0, 0, 1, 1, 0, 0, 1, 1, 0, 0]]
[[  0.         126.98689114]
 [ 14.9702437    0.        ]]
Is EF1 allocation?  False
Allocation :  [[1, 1, 0, 1, 0, 1, 1, 0, 0, 0], [0, 0, 1, 0, 1, 0, 0, 1, 1, 1]]
[[ 0.         11.39284669]
 [60.30953183  0.        ]]
Is EF1 allocation?  False
Allocation :  [[0, 1, 0, 0, 0, 0, 1, 1, 0, 1], [1, 0, 1, 1, 1, 1, 0, 0, 1, 0]]
[[  0.         -41.95967719]
 [ 16.88636465   0.        ]]
Is EF1 allocation?  False
Allocation :  [[

Allocation :  [[1, 0, 1, 0, 0, 1, 1, 1, 1, 1], [0, 1, 0, 1, 1, 0, 0, 0, 0, 0]]
[[   0.          419.47665217]
 [-198.4530004     0.        ]]
Is EF1 allocation?  False
Allocation :  [[0, 1, 1, 0, 0, 1, 1, 1, 0, 0], [1, 0, 0, 1, 1, 0, 0, 0, 1, 1]]
[[  0.          66.78063885]
 [-75.66011256   0.        ]]
Is EF1 allocation?  False
Allocation :  [[0, 0, 0, 1, 1, 1, 0, 1, 0, 1], [1, 1, 1, 0, 0, 0, 1, 0, 1, 0]]
[[  0.         111.105842  ]
 [227.20843589   0.        ]]
Is EF1 allocation?  True
Allocation :  [[0, 1, 1, 1, 0, 1, 1, 1, 0, 0], [1, 0, 0, 0, 1, 0, 0, 0, 1, 1]]
[[  0.         343.60261933]
 [  5.85607727   0.        ]]
Is EF1 allocation?  False
Allocation :  [[1, 1, 0, 0, 1, 1, 1, 1, 1, 0], [0, 0, 1, 1, 0, 0, 0, 0, 0, 1]]
[[   0.          306.83610327]
 [-140.25440517    0.        ]]
Is EF1 allocation?  False
Allocation :  [[0, 0, 0, 0, 0, 0, 1, 0, 1, 1], [1, 1, 1, 1, 1, 1, 0, 1, 0, 0]]
[[   0.         -248.48650195]
 [ 316.89409019    0.        ]]
Is EF1 allocation?  False
Alloc

Allocation :  [[1, 1, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 1, 1, 1, 1, 1, 1, 1, 0]]
[[   0.         -139.75961663]
 [ 536.82530477    0.        ]]
Is EF1 allocation?  False
Allocation :  [[1, 0, 1, 0, 1, 1, 1, 0, 1, 0], [0, 1, 0, 1, 0, 0, 0, 1, 0, 1]]
[[  0.          48.42082225]
 [-43.40720811   0.        ]]
Is EF1 allocation?  False
Allocation :  [[1, 1, 0, 0, 0, 0, 0, 0, 1, 1], [0, 0, 1, 1, 1, 1, 1, 1, 0, 0]]
[[   0.         -169.90763891]
 [ 127.27775596    0.        ]]
Is EF1 allocation?  False
Allocation :  [[1, 0, 1, 0, 0, 1, 1, 0, 1, 1], [0, 1, 0, 1, 1, 0, 0, 1, 0, 0]]
[[   0.          432.69936392]
 [-133.13380191    0.        ]]
Is EF1 allocation?  False
Allocation :  [[1, 1, 1, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 1, 1, 1, 1, 1]]
[[  0.          54.14662681]
 [159.79539668   0.        ]]
Is EF1 allocation?  False
Allocation :  [[1, 0, 1, 1, 1, 0, 0, 0, 1, 1], [0, 1, 0, 0, 0, 1, 1, 1, 0, 0]]
[[   0.           42.02708244]
 [-193.2849176     0.        ]]
Is EF1 allocation?  False


Is EF1 allocation?  True
Allocation :  [[0, 1, 0, 0, 1, 1, 1, 0, 0, 0], [1, 0, 1, 1, 0, 0, 0, 1, 1, 1]]
[[  0.         -30.1440809 ]
 [148.05331021   0.        ]]
Is EF1 allocation?  True
Allocation :  [[0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [1, 1, 1, 1, 1, 1, 0, 1, 1, 1]]
[[   0.         -473.74597872]
 [ 440.3518223     0.        ]]
Is EF1 allocation?  False
Allocation :  [[0, 0, 1, 0, 0, 0, 0, 0, 1, 0], [1, 1, 0, 1, 1, 1, 1, 1, 0, 1]]
[[   0.         -168.32783146]
 [ 385.85615127    0.        ]]
Is EF1 allocation?  False
Allocation :  [[1, 1, 1, 0, 0, 1, 1, 0, 0, 1], [0, 0, 0, 1, 1, 0, 0, 1, 1, 0]]
[[  0.         148.67425208]
 [-23.23891571   0.        ]]
Is EF1 allocation?  True
Allocation :  [[1, 1, 1, 1, 1, 0, 1, 0, 1, 0], [0, 0, 0, 0, 0, 1, 0, 1, 0, 1]]
[[   0.          117.69160228]
 [-105.99696032    0.        ]]
Is EF1 allocation?  False
Allocation :  [[0, 1, 0, 1, 0, 0, 1, 1, 0, 1], [1, 0, 1, 0, 1, 1, 0, 0, 1, 0]]
[[   0.         -103.74845596]
 [ 127.39374531    0.        ]]
Is 

Is EF1 allocation?  False
Allocation :  [[1, 0, 0, 1, 0, 1, 1, 1, 0, 0], [0, 1, 1, 0, 1, 0, 0, 0, 1, 1]]
[[  0.         -88.29010371]
 [ 77.2257887    0.        ]]
Is EF1 allocation?  False
Allocation :  [[1, 1, 0, 1, 0, 0, 0, 0, 1, 0], [0, 0, 1, 0, 1, 1, 1, 1, 0, 1]]
[[   0.         -102.89335369]
 [  13.15889607    0.        ]]
Is EF1 allocation?  False
Allocation :  [[1, 1, 0, 0, 0, 0, 1, 0, 1, 1], [0, 0, 1, 1, 1, 1, 0, 1, 0, 0]]
[[ 0.         89.71505134]
 [ 1.78312765  0.        ]]
Is EF1 allocation?  True
Allocation :  [[1, 0, 0, 0, 0, 1, 0, 0, 0, 1], [0, 1, 1, 1, 1, 0, 1, 1, 1, 0]]
[[   0.         -173.27659379]
 [ 276.4191494     0.        ]]
Is EF1 allocation?  False
Allocation :  [[0, 1, 0, 0, 0, 0, 1, 0, 0, 1], [1, 0, 1, 1, 1, 1, 0, 1, 1, 0]]
[[   0.         -167.75083653]
 [ 283.15860029    0.        ]]
Is EF1 allocation?  False
Allocation :  [[1, 1, 1, 1, 1, 0, 1, 1, 0, 1], [0, 0, 0, 0, 0, 1, 0, 0, 1, 0]]
[[   0.          460.84669046]
 [-210.58560748    0.        ]]
Is EF

Allocation :  [[1, 1, 0, 1, 1, 0, 1, 1, 0, 0], [0, 0, 1, 0, 0, 1, 0, 0, 1, 1]]
[[   0.          131.41960641]
 [-122.1169495     0.        ]]
Is EF1 allocation?  False
Allocation :  [[1, 0, 0, 0, 1, 1, 0, 1, 0, 1], [0, 1, 1, 1, 0, 0, 1, 0, 1, 0]]
[[ 0.         28.79465992]
 [ 2.26757673  0.        ]]
Is EF1 allocation?  True
Allocation :  [[0, 1, 1, 0, 1, 1, 0, 0, 1, 1], [1, 0, 0, 1, 0, 0, 1, 1, 0, 0]]
[[  0.         226.18137435]
 [-19.81936103   0.        ]]
Is EF1 allocation?  True
Allocation :  [[1, 0, 0, 1, 1, 1, 1, 0, 1, 0], [0, 1, 1, 0, 0, 0, 0, 1, 0, 1]]
[[  0.         264.13853535]
 [-24.23019707   0.        ]]
Is EF1 allocation?  False
Allocation :  [[1, 0, 1, 0, 1, 0, 0, 1, 1, 1], [0, 1, 0, 1, 0, 1, 1, 0, 0, 0]]
[[ 0.         74.54342133]
 [-3.25526577  0.        ]]
Is EF1 allocation?  False
Allocation :  [[1, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 1, 1, 1, 1, 1, 1, 1, 0, 1]]
[[   0.         -263.3654147 ]
 [ 393.04137583    0.        ]]
Is EF1 allocation?  False
Allocation :  [[1,

In [92]:
a = np.array([[1, 2, 3],[3, -4, 5]])

In [97]:
np.sum(a<0)

1

In [93]:
bool(sum(a<0))

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()